Before you turn this problem in, make sure everything runs as expected. **Restart the kernel and run all cells** (in the menubar, select Kernel$\rightarrow$Restart & Run All).

Make sure you fill in any place that says `YOUR CODE HERE ???` or "YOUR ANSWER HERE". Code cells are autograded, and if the test cells don't run all the way to the l that they print "Tests Passed (# points)"

# CSCI 3155: Assignment 4

Topics Covered: Operations on inductive definitions.

__Name__: Maura Kieft

## Problem 1: Pattern Matching, Inductive Structures (40 points)

This assignment asks you to implement `hasKey` and `deleteKey` operations on a Binary Search Tree (BST) in order to implement `deleteKey`. We have already seen how to  do some basic tasks on BSTs such as `isBST` and `insert` (lecture on Sept. 18, recitation on Sept. 21).

### 1A: hasKey (5 points)

Implement a `hasKey` function on binary search trees that returns `true` if a key `k` is present and `false` otherwise.

In [10]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"Tests Passed (1 point)")
    else print(s"Tests Passed ($points points)") 
}

passed: (points: Int)Unit


In [2]:
sealed trait NumTree
case object Leaf extends NumTree
case class Node(j: Int, left: NumTree, right: NumTree ) extends NumTree

def hasKey(t: NumTree, k: Int): Boolean ={
    t match{
        case Leaf => false
        case Node(j,left,right) => {
            if( j == k){
                true
            }
            else if( j > k){
                hasKey(left,k)
            }
            else{
                hasKey(right,k)
            }
        }
            
        }
            
    }

defined trait NumTree
defined object Leaf
defined class Node
hasKey: (t: NumTree, k: Int)Boolean


In [3]:
// BEGIN TESTS
val t1 = Leaf
assert(hasKey(t1, 20) == false, "Test 1 failed: hasKey(Leaf, 20) should be false")

val t2 = Node(10, Node(8, Leaf, Node(9, Leaf, Leaf)), Node(12, Node(11, Leaf, Leaf), Leaf))
assert(hasKey(t2, 10), "Test 2 failed hasKey(t2, 10)")
assert(hasKey(t2, 11), "Test 3 failed hasKey(t2, 11)")
assert(!hasKey(t2, 7), "Test 4 failed !hasKey(t2, 7)")

val t3 = Node(20, t2, Node(25, Node(18, Leaf, Leaf), Leaf))
assert(!hasKey(t3, 7), "Test 5 failed !hasKey(t3, 7)")
assert(hasKey(t3, 12), "Test 6 failed hasKey(t3, 12)")

passed(5)
// END TESTS

Tests Passed (5 points)

null

### 1B: deleteMin (15 points)

Implement a function `deleteMin(t: NumTree): (NumTree, Int)` that deletes the minimum element of a binary search tree `t` and returns a tuple of two things: (a) the new tree with the deletion and (b) the minimum of the old tree. 

In [208]:
def deleteMin(t: NumTree): (NumTree, Int) =  {
    require(t != Leaf)
    def findMinElementHelper(t: NumTree): Int = t match {
    case Leaf => Int.MaxValue
    case Node(value, left, right) => {
        val v1 = findMinElementHelper(left) // min of left tree 
        val v2 = findMinElementHelper(right) //min of right tree
        return List(value, v1, v2).min
        
        }
    
    }

    val minVal = findMinElementHelper(t)
    //println("Min Element: ", minVal)
    def removeHelper(t:NumTree, minVal: Int): NumTree = {
            t match{
                case Leaf => throw new IllegalArgumentException
                case Node(value,Leaf,Leaf) => Leaf
                case Node(value,Leaf,right)=> right
                case Node(value,left,right)=>{
                    if(minVal < value){
                        //println("minval < value: value= ", value)
                        Node(value,removeHelper(left,minVal),right)
                    }
                    else if(minVal>value){
                        //println("minVal > value: value=", value)
                        Node(value,left,removeHelper(right,minVal))
                    }
                    else {
                        
                        val min = findMinElementHelper(right) 
                        println("Right min: ",min)
                        Node(value,left,removeHelper(right,min))
                    }
                    
                }
            
            }
    }

    (removeHelper(t,minVal),minVal)
    
}



deleteMin: (t: NumTree)(NumTree, Int)


In [209]:
// BEGIN TESTS

// TEST 1
val t1 = Node(10, Leaf, Leaf )
val t1r = Leaf
val (t1res, minval1) = deleteMin(t1)
assert((t1res, minval1) == (t1r, 10), s"Test 1 failed: expected $t1r, obtained: ${(t1res, minval1)})")

passed(5)

Tests Passed (5 points)

null

In [210]:
// TEST 2
val t2 = Node(10, Node(8, Leaf, Node(9, Leaf, Leaf)), Node(12, Node(11, Leaf, Leaf), Leaf))
val t2r = Node(10, Node(9, Leaf, Leaf), Node(12, Node(11, Leaf, Leaf), Leaf))
val (t2res, minval2) = deleteMin(t2)
assert((t2res, minval2) == (t2r, 8), s"Test 2 failed: expected $t2r, obtained: (${(t2res, minval2)})")

// Test 3
val t3 = Node(20, t2, Node(25, Node(18, Leaf, Leaf), Leaf))
val t3r = Node(20, t2r, Node(25, Node(18, Leaf, Leaf), Leaf))
val (t3res, minval3) = deleteMin(t3)
assert((t3res, minval3) == (t3r, 8), s"Test 3 failed: expected $t3r, obtained: (${(t3res, minval3)})")

// Test 4

val t4 = Node(20, Node(10, Node(5, Node(2, Node(1, Leaf, Leaf), Node(3, Leaf, Leaf)), Leaf), Leaf), Leaf)
val t4r = Node(20, Node(10, Node(5, Node(2, Leaf, Node(3, Leaf, Leaf)), Leaf), Leaf), Leaf)
val (t4res, minval4) = deleteMin(t4)
assert((t4res, minval4) == (t4r, 1), s"Test 4 failed: expected $t4r, obtained: (${(t4res, minval4)})")

passed(10)
// END TESTS

Tests Passed (10 points)

null

### 1C: deleteKey (20 points)

Now implement a function `deleteKey(j: Int)` that deletes a given key from the BST. Assume that the key `j` to be deleted will be part of the BST.

Recall how to do BST deletion:

- First recursively get to the node to be deleted.
- If one of its children is a `Leaf`, simply replace the node by the other child.
- If both its children are non leaf, use `deleteMin` on the right child and 
   replace the node's value by the integer value returned by `deleteMin`, and the right child of the
   node by the resulting tree of the `deleteMin`.

In [211]:
def deleteKey(t: NumTree, j: Int): NumTree ={
    // YOUR CODE HERE
    t match {
            case Leaf => Leaf
            case Node(value,Leaf,Leaf)=> Leaf
            //case Node(value,Leaf,right) => right
            //case Node(value, left, Leaf)=> left
            case Node(value,left,right) => {
                if (j == value){
                    //println("j==value", "j=",j, "val=",value)
                    val (restTree, newVal) = deleteMin(right)
                    //println("Changing right child: ", "NewVal= ",newVal, "restTree= ",restTree)
                    if( left == Leaf){
                        right
                    }
                    else{
                    (Node(newVal, left, restTree))
                    }
                }
                else if(value > j){
                    //println("value > j", "value= ", value, "j= ",j)
                    Node(value,deleteKey(left,j),right)
                    //deleteKey(left,j)
                }
                else{
                    //println("value < j")
                    Node(value,left,deleteKey(right,j))
                    //deleteKey(right,j)
                }
            }
        }
    } 


deleteKey: (t: NumTree, j: Int)NumTree


In [205]:
// BEGIN TESTS
// TEST 1
val t1 = Node(10, Leaf, Leaf )
assert(deleteKey(t1, 10) == Leaf, "Test 1 failed: deleteKey(t1, 10) == Leaf")

passed(5)

Tests Passed (5 points)

null

In [206]:
// TEST 2
val t2 = Node(10, Node(8, Leaf, Node(9, Leaf, Leaf)), Node(15, Leaf, Leaf))
assert(deleteKey(t2, 8) == Node(10, Node(9, Leaf, Leaf), Node(15, Leaf, Leaf)), "Test 2 failed" )
passed(5)

Tests Passed (5 points)

null

In [207]:
// TEST 3
val t3 = Node(25, t2, Node(100, Leaf, Node(120, Leaf, Leaf)))
assert(deleteKey(t3, 25) == Node(100, t2, Node(120, Leaf, Leaf)), "Test 3 failed")
// Test 4
val t4 = Node(25, Leaf, Node(40, Node(35, Leaf, Leaf), Node(50, Node(49, Leaf, Leaf), Leaf)))
val t4_expected = Node(25, Leaf, Node(49, Node(35, Leaf, Leaf), Node(50, Leaf, Leaf)))
val t4_result = deleteKey(t4, 40)
assert(t4_result == t4_expected, "Test 4 failed")
// Test 5 
val t5_expected= Node(40,  Node(35, Leaf, Leaf), Node(50, Node(49, Leaf, Leaf), Leaf))
val t5_result = deleteKey(t4, 25)
assert(t5_result == t5_expected, "Test 5 Failed")
// Test 6
val t6h1 = Node(50, Node(45, Leaf, Leaf), Node(75, Node(73, Node(71, Node(69, Node(67, Node(64, Leaf, Leaf), Node(68, Leaf, Leaf)),Leaf), Leaf), Leaf), Leaf))
val t6 = Node(25, t2, Node(100, t6h1, Leaf))
val t6h2 = Node(64, Node(45, Leaf, Leaf), Node(75, Node(73, Node(71, Node(69, Node(67, Leaf, Node(68, Leaf, Leaf)),Leaf), Leaf), Leaf), Leaf))
val t6_expected = Node(25, t2, Node(100, t6h2, Leaf))
val t6_result = deleteKey(t6, 50)
assert(t6_result == t6_expected, "Test 6 failed")

passed(10)
// END TESTS

Tests Passed (10 points)

null